Dans ce notebook, vous apprendrez à classer des images de chats et de chiens en utilisant un transfert learning à partir d'un réseau pré-entraîné sur le jeu de données [ImageNet](https://www.image-net.org/update-mar-11-2021.php).

# Importation des données

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os

import urllib.request
import zipfile

import torch
from torchvision import datasets, transforms
from torchsummary import summary

# Chargement des données

Téléchargement des données

In [ ]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
urllib.request.urlretrieve(_URL, "cats_and_dogs.zip")

Unzip du fichier.

In [ ]:
with zipfile.ZipFile("cats_and_dogs.zip", 'r') as zip_ref:
    zip_ref.extractall()

In [ ]:
PATH = "cats_and_dogs_filtered"

Sauvegarde des PATH

In [ ]:
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

# Fixer les hyperparamètres

In [ ]:
BATCH_SIZE = 32
IMG_SIZE = (128, 128)

# Initialiser les générateurs

Pour le jeu d'entraînement.

Utilisez la fonction `Compose` de `transforms` avec en paramètre `Resize` et `ToTensor`.

In [ ]:
transform = None

Utilisez `ImageFolder` de `datasets` avec comme paramètre le path du dossier `train_dir` et `transform` initialisé précédemment.

In [ ]:
dataset_train = None

Utilisez `DataLoader` pour initialiser le générateur de données avec `shuffle` à True et une taille de batch à 32.

In [ ]:
dataloader_train = None

Visualisez ce que ça donne.

In [ ]:
for x, y in dataloader_train:
    print(x.shape)
    print(y.shape)
    plt.imshow(np.transpose(x[0, :, :, :].squeeze(), (1, 2, 0)), cmap='gray')
    plt.show()
    break

Pour le jeu de validation.

Faire de même pour la validation.

In [ ]:
dataset_test = None

dataloader_test = None

Visualisez le résultat.

In [ ]:
for x, y in dataloader_test:
    print(x.shape)
    print(y.shape)
    plt.imshow(np.transpose(x[0, :, :, :].squeeze(), (1, 2, 0)), cmap='gray')
    plt.show()
    break

#  Charger un modèle pré-entraîné

In [ ]:
IMG_SHAPE = (3,) + IMG_SIZE

Vous allez utiliser un modèle pré-entrainé sur imagenet que vous allez finetuné pour différencier des photos de chats et de chiens.

Vous avez le choix entre plusieurs modèles.

Vous pouvez jeter un oeil [ici](https://pytorch.org/vision/stable/models.html#classification).

Dans ce notebook vous allez utilisé mobilenet_V2 car elle est assez léger ce qui réduira le temps de calcul.



Utilisez la fonction `hub.load` pour charger le modèle en question.

In [ ]:
# Create the base model from the pre-trained model MobileNet V2
model = None

Regardez l'architecture du modèle en utilisant `summary`.

Si votre notebook utilise le GPU vous aurez peut être besoin de lui spécifier `device='cpu'` en paramètre.

In [ ]:
None

Appliquez le modèle sur `x`.

In [ ]:
feature_batch = None

Regadrez la dimension de feature_batch.

In [ ]:
feature_batch.shape

On voit qu'il a 1000 sortie ce qui correspond au 1000 classes de ImageNet.

Pour notre application il nous faudra 2 classes, une pour les chiens, une pour les chats.

Le modèle est décomposé en une partie feature et une partie classifier.

In [ ]:
model

La partie classifier est décomposé en une couche de drop out et une couche linéaire.

In [ ]:
model.classifier

La couche reçoit en entrée 1280 variables.

In [ ]:
model.classifier[1].in_features

On le remplace par une couche qui reçoit la même chose mais qui n'aura qu'une neurone en sortie car nous allons faire la différence entre un chien et un chat.

Utilisez `Linear` avec 2 neurones pour remplacer le `Dropout`.

Utilisez `LogSoftmax` pour remplacer le `Linear` avec 1000 classes.

In [ ]:
model.classifier[0] = None
model.classifier[1] = None

La modification a bien été effectué.

In [ ]:
model.classifier

Visualisez le modèle maintenant en utilisant `summary`.

In [ ]:
None

Fixer le modèle de base pour que les poids ne soient pas mis-à-jour.

Utiliser la méthode `requires_grad` de chacun des paramètres `param` pour changer la valeur de `True` à `False`.

In [ ]:
for param in model.features.parameters():
    None

Vous pouvez maintenant regarder avec la fonction `summary` le nombre de parmètres entrainable qui ne correspond qu'à la dernière ligne.

In [ ]:
None

# Les fonctions d'entraînement

In [ ]:
def number_of_good_prediction(prediction:float, target:int):
  one_hot_prediction = np.argmax(prediction, axis=1)
  return np.sum(one_hot_prediction == target)

In [ ]:
def save_model(model, path):
    torch.save(model.state_dict(), path)

In [ ]:
def step(model:torch.nn.Sequential,
         opt:torch.optim,
         criterion:torch.nn.modules.loss,
         x_train:torch.Tensor,
         y_train:torch.Tensor,
         metric_function)->tuple:
  """
  Executes a single training step for a PyTorch model.
  This function performs a forward pass to compute the model's predictions, calculates
  the loss between predictions and actual target values, computes gradients for each
  model parameter, and updates the parameters using the optimizer.

  Args:
      model (torch.nn.Sequential): The PyTorch model to train.
      optimizer (torch.optim.Optimizer): Optimizer used to update the model's parameters.
      criterion (torch.nn.modules.loss._Loss): Loss function used to compute the error.
      x_train (torch.Tensor): Input training data (features).
      y_train (torch.Tensor): Ground truth labels or target values for the training data.
  Returns:
      tuple: The updated model and the computed loss for the current step.
  """

  # Réinitialisez les gradients d'optimizer à zéro avec la méthode 'zero_grad'
  opt.zero_grad()

  # Calculez les prédiction sur le jeu d'entraînement avec la méthode 'froward'
  prediction = model.forward(x_train)

  # Calculez l'erreur de prédiction avec 'criterion'
  loss = criterion(prediction, y_train)

  performance = metric_function(prediction.detach().numpy(), y_train.detach().numpy())

  # Calculez les gradients avec la méthode 'backward'
  loss.backward()

  # Mettre à jour les paramètres du modèle avec la méthode 'step'
  opt.step()

  return model, loss, performance

In [ ]:
def fit(model, optimizer, criterion, epoch, trainloader, testloader, metric_function):
    epoch = epoch
    history_train_loss = []
    history_test_loss = []
    history_train_metrics = []
    history_test_metrics = []

    reference_performance = 0

    for e in range(epoch) :

      train_loss_batch = 0
      test_loss_batch = 0
      train_metric_batch = 0
      test_metric_batch = 0

      for images, labels in trainloader:

        # mise à jour des poids avec la fonction 'step'
        model, train_loss, train_performance = step(model, optimizer, criterion, images, labels, metric_function)

        train_loss_batch += train_loss.detach().numpy()

        train_metric_batch += train_performance


      for images, labels in testloader:

        prediction = model.forward(images)

        test_loss = criterion(prediction, labels)

        test_metric_batch += metric_function(prediction.detach().numpy(), labels.detach().numpy())

        test_loss_batch += test_loss.detach().numpy()

      train_loss_batch /= len(trainloader.sampler)
      test_loss_batch /= len(testloader.sampler)

      train_metric_batch /= len(trainloader.sampler)
      test_metric_batch /= len(testloader.sampler)

      # Sauvegarde des coûts d'entraînement avec append
      history_train_loss = np.append(history_train_loss, train_loss_batch)
      history_test_loss = np.append(history_test_loss, test_loss_batch)

      # Sauvegarde des coûts d'entraînement avec append
      history_train_metrics = np.append(history_train_metrics, train_metric_batch)
      history_test_metrics = np.append(history_test_metrics, test_metric_batch)

      print('train_loss : '+str(np.squeeze(train_loss_batch))+ ' test_loss : '+str(np.squeeze(test_loss_batch)))
      print('train_metric : '+str(np.squeeze(train_metric_batch))+ ' test_metric : '+str(np.squeeze(test_metric_batch)))
      print('-------------------------------------------------------------------------------------------------')

      if test_metric_batch > reference_performance:
        reference_performance = test_metric_batch
        save_model(model, f'best_model_{test_metric_batch}.pth')

    return model, history_train_loss, history_test_loss, history_train_metrics, history_test_metrics


# Compiler le modèle

Initialisez `criterion` avec la fonction `NLLLoss`.

Initialisez `optimizer` avec `Adam` et un learning rate de 0.001.

In [ ]:
criterion = None
optimizer = None

# Entraînement du modèle

Utilisez la fonction `fit` pour entraîner le modèle.

In [ ]:
epoch = 5

model, history_train_loss, history_test_loss, history_train_metrics, history_test_metrics = None

In [ ]:
plt.plot(np.arange(epoch), history_train_loss, label='train loss')
plt.plot(np.arange(epoch), history_test_loss, label='test loss')
plt.xlabel('Epochs')
plt.ylabel('loss')
plt.legend(loc='upper left')
plt.legend()
plt.show()

In [ ]:
plt.plot(history_train_metrics, label='train accuracy')
plt.plot(history_test_metrics, label='test accuracy')
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(loc='upper left')
plt.show()

# Fine tuning

Entraîner tous les paramètres du modèle.

Utiliser la méthode `requires_grad` de chacun des paramètres `param` pour changer la valeur de `False` à `True`.

In [ ]:
for param in model.features.parameters():
    None

Utilisez `summary` pour visualiser le nombre de paramètre entraînable du modèle maintenant.

In [ ]:
None

Compiler le nouveau modèle

In [ ]:
criterion = None
optimizer = None

Continuer l'entraînement du modèle avec toutes les couches entrainables en utilisant la fonction `fit`.

In [ ]:
epoch = 5

model, history_train_loss_2, history_test_loss_2, history_train_metrics_2, history_test_metrics_2 = None

Concatenation des vecteurs de résultats.

In [ ]:
loss = np.concatenate([history_train_loss, history_train_loss_2])
val_loss = np.concatenate([history_test_loss, history_test_loss_2])

perf = np.concatenate([history_train_metrics, history_train_metrics_2])
val_perf = np.concatenate([history_test_metrics, history_test_metrics_2])

Visualiser les performances

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(perf, label='Training Accuracy')
plt.plot(val_perf, label='Validation Accuracy')
#plt.ylim([0.8, 1])
plt.plot([epoch-1,epoch-1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
#plt.ylim([0, 1.0])
plt.plot([epoch-1,epoch-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()